In [15]:

# Import necessary libraries

import os
import nest_asyncio
import numpy as np
import cv2
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import StreamingResponse
from io import BytesIO
import uvicorn


In [16]:

# Initialize FastAPI app

app = FastAPI()



In [17]:
TEMPLATE_DIR = os.path.abspath("templates")  
TEMPLATES = {
    "apple": [os.path.join(TEMPLATE_DIR, "apple1.jpg"), os.path.join(TEMPLATE_DIR, "apple2.jpg")],
    "banana": [os.path.join(TEMPLATE_DIR, "banana1.jpg"), os.path.join(TEMPLATE_DIR, "banana2.jpg")]
}

In [18]:
# 🛠️ DEBUG: Check if template images exist
for fruit, template_list in TEMPLATES.items():
    for template in template_list:
        if not os.path.exists(template):
            print(f"⚠️ Missing template: {template}")

⚠️ Missing template: /Users/user/Documents/GitHub/image-detection-app/templates/apple1.jpg
⚠️ Missing template: /Users/user/Documents/GitHub/image-detection-app/templates/apple2.jpg
⚠️ Missing template: /Users/user/Documents/GitHub/image-detection-app/templates/banana1.jpg
⚠️ Missing template: /Users/user/Documents/GitHub/image-detection-app/templates/banana2.jpg


In [19]:

def detect_fruits_color(image: np.ndarray):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Apple (Red)
    lower_red1, upper_red1 = np.array([0, 100, 100]), np.array([10, 255, 255])
    lower_red2, upper_red2 = np.array([160, 100, 100]), np.array([180, 255, 255])
    mask_apples = cv2.inRange(hsv, lower_red1, upper_red1) + cv2.inRange(hsv, lower_red2, upper_red2)

    # Banana (Yellow)
    lower_yellow, upper_yellow = np.array([20, 100, 100]), np.array([35, 255, 255])
    mask_bananas = cv2.inRange(hsv, lower_yellow, upper_yellow)

    kernel = np.ones((5, 5), np.uint8)
    mask_apples = cv2.morphologyEx(mask_apples, cv2.MORPH_OPEN, kernel)
    mask_bananas = cv2.morphologyEx(mask_bananas, cv2.MORPH_OPEN, kernel)

    contours_apples, _ = cv2.findContours(mask_apples, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_bananas, _ = cv2.findContours(mask_bananas, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    apple_count = sum(1 for c in contours_apples if cv2.contourArea(c) > 1000)
    banana_count = sum(1 for c in contours_bananas if cv2.contourArea(c) > 1000)

    return image, {"apple": apple_count, "banana": banana_count}


def detect_fruits_template(image: np.ndarray, templates: dict):
    detected_image = image.copy()
    apple_count, banana_count = 0, 0

    # Loop through each fruit type and its templates
    for fruit, template_list in templates.items():
        for template_path in template_list:
            if not os.path.exists(template_path):
                print(f"⚠️ Template missing: {template_path}")
                continue

            # Load the template image in grayscale
            template_img = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
            if template_img is None:
                print(f"❌ Could not read {template_path}")
                continue

            # Resize the template at different scales
            for scale in np.linspace(0.8, 1.2, 5):
                resized_template = cv2.resize(template_img, None, fx=scale, fy=scale)

                # Perform template matching
                res = cv2.matchTemplate(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), resized_template, cv2.TM_CCOEFF_NORMED)

                # Set threshold for detection confidence
                threshold = 0.7
                loc = np.where(res >= threshold)

                # Loop through all detected locations
                for pt in zip(*loc[::-1]):  # loc gives coordinates in reversed order
                    # Draw a bounding box around the detected fruit
                    top_left = pt
                    bottom_right = (pt[0] + resized_template.shape[1], pt[1] + resized_template.shape[0])
                    cv2.rectangle(detected_image, top_left, bottom_right, (0, 255, 0), 3)

                    # Add label with the name of the fruit (e.g., "Apple" or "Banana")
                    label = f"{fruit.capitalize()}"
                    label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                    label_x = top_left[0]
                    label_y = top_left[1] - 10

                    # Draw the label background (white rectangle behind the label)
                    cv2.rectangle(detected_image, (label_x, label_y - label_size[1]), 
                                  (label_x + label_size[0], label_y + 5), (0, 255, 0), -1)

                    # Draw the label text in white
                    cv2.putText(detected_image, label, (label_x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

                    # Count the detected fruits
                    if fruit == "apple":
                        apple_count += 1
                    elif fruit == "banana":
                        banana_count += 1

    return detected_image, {"apple": apple_count, "banana": banana_count}

In [20]:
# API endpoint to upload images and detect if they are apples or not 
@app.post("/detect_fruits")
async def detect_fruits(file: UploadFile = File(...)):
    try:
        # Read image file
        image_bytes = await file.read()
        
        # Convert to NumPy array
        image_array = np.frombuffer(image_bytes, np.uint8)
        
        # Decode image
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
        
        if image is None:
            print("Error: Could not decode image")
            return {"error": "Invalid image file"}

        # Set up template paths (these should be real paths to template images)
        templates = {
            "apple": ["apple_template1.jpg", "apple_template2.jpg"],
            "banana": ["banana_template1.jpg", "banana_template2.jpg"]
        }

        # Perform fruit detection using both methods
        _, fruit_counts_color = detect_fruits_color(image)
        image_template, fruit_counts_template = detect_fruits_template(image, templates)

        # Combine the results (just summing counts for apples and bananas)
        total_counts = {
            "apple": fruit_counts_color["apple"] + fruit_counts_template["apple"],
            "banana": fruit_counts_color["banana"] + fruit_counts_template["banana"]
        }

        # Encode image with bounding boxes
        _, encoded_image = cv2.imencode(".jpg", image_template)
        
        # Return the image with bounding boxes as a response
        return {
            "apple_count": total_counts["apple"],
            "banana_count": total_counts["banana"],
            "image": StreamingResponse(BytesIO(encoded_image.tobytes()), media_type="image/jpeg")
        }

    except Exception as e:
        print(f"Error: {e}")
        return {"error": str(e)}

In [21]:
# Runnin the API server
nest_asyncio.apply()  # Allows FastAPI to run inside Jupyter

uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [26475]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53655 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53655 - "GET /openapi.json HTTP/1.1" 200 OK
⚠️ Template missing: apple_template1.jpg
⚠️ Template missing: apple_template2.jpg
⚠️ Template missing: banana_template1.jpg
⚠️ Template missing: banana_template2.jpg
INFO:     127.0.0.1:53658 - "POST /detect_fruits HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/fastapi/encoders.py", line 324, in jsonable_encoder
    data = dict(obj)
           ^^^^^^^^^
TypeError: 'async_generator' object is not iterable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/fastapi/encoders.py", line 329, in jsonable_encoder
    data = vars(obj)
           ^^^^^^^^^
TypeError: vars() argument must have __dict__ attribute

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/uvicorn/middleware/proxy_header

references for the code used: 
National Film Board of Canada. (2017, April 12). Trick or Treaty? - Alanis Obomsawin (2014) [HD, 52 min] [Video]. YouTube. https://www.youtube.com/watch?v=tc5-wp7D9ko 
most of image detection video served as references point for my code
